In [ ]:
import torch
import torch.linalg as tla
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

We want to simulate the PDE
$$ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0 $$
on the domain $\Omega = [-1, 1]$, with initial and boundary conditions
$$\begin{align*}
u(x, 0) &= -\sin(\pi x) \\
u(-1, t) &= 0 \\
u(1, t) &= 0 \\
\end{align*}$$

For context, here is what the solution is supposed to look like: https://www.youtube.com/watch?v=bDNXNGpYj0c.

One way we can solve this PDE this is to *learn* the function $\hat{u}(x, t)$ that will approximate the solution, where
$$ \hat{u} : \mathbb{R}^2 \to \mathbb{R} $$
is a function parameterized by a feed-forward neural network.

In [ ]:
# We will learn the function from t=0 to t=0.75
t0 = 0
t1 = 0.75

# Initial condition in time
def u0(x):
    return -torch.sin(torch.pi * x)

In [ ]:
x = torch.linspace(-1, 1, 50)
plt.plot(x, u0(x), '-')

In [ ]:
class FCN(torch.nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        N = 16
        self.fc_layers = torch.nn.Sequential(
            torch.nn.Linear(2, N), torch.nn.Tanh(),
            torch.nn.Linear(N, N), torch.nn.Tanh(),
            torch.nn.Linear(N, N), torch.nn.Tanh(),
            torch.nn.Linear(N, 1)
        )

    def forward(self, u, t):
        return self.fc_layers(torch.column_stack((u, t))).flatten()

def plot_solution(u_hat, t0, t1, grid_spacing=128, time_steps=50):
    fig, ax = plt.subplots()

    x = torch.linspace(-1, 1, grid_spacing)
    t = torch.linspace(t0, t1, time_steps)
    line, = ax.plot(x, u_hat(x, torch.ones_like(x) * t0).detach())

    def init():
        return line,

    def update(t_):
        line.set_data(x, u_hat(x, torch.ones_like(x) * t_).detach())
        return line,

    anim = FuncAnimation(fig, update, frames=t, init_func=init, blit=True, interval=40)
    plt.close(fig)
    return HTML(anim.to_html5_video())

In [ ]:
u_hat = FCN()
plot_solution(u_hat, t0, t1)

Nothing interesting happens, but we haven't trained anything!  First, lets learn the initial condition by just optimizing $$\ell = \|\hat{u}(x, 0) - u(x, 0)\|$$ over some discrete samples $x$.

In [ ]:
t = torch.zeros_like(x)
opt = torch.optim.Adam(u_hat.parameters(), lr=0.005)

for i in range(1_000):
    opt.zero_grad()
    ell = tla.norm(u_hat(x,t).flatten() - u0(x))
    ell.backward()
    opt.step()

    if i % 50 == 0:
        print(i, ell.item())

In [ ]:
plot_solution(u_hat, t0, t1)

That works, but notice how the boundary conditions are only respected at $t=0$.  Lets modify our loss function to penalize the boundary terms, giving us
$$\ell = \|\hat{u}(x, 0) - u(x, 0)\| + \|\hat{u}(-1, t)\| + \|\hat{u}(1, t)\|,$$
over now some discrete $x$, $t$.

In [ ]:
# space, time for initial conditions
x_ic = torch.linspace(-1, 1, 50)
t_ic = torch.zeros_like(x)

# space, time for boundary conditions
t_bc = torch.linspace(t0, t1, 50)
x_l_bc = torch.ones_like(t_bc) * -1
x_r_bc = torch.ones_like(t_bc) * 1

opt = torch.optim.Adam(u_hat.parameters(), lr=0.001)

for i in range(1_000):
    # fill me
    opt.zero_grad()
    ell_ic = tla.norm(u_hat(x_ic, t_ic) - u0(x))
    ell_bc_l = ...
    ell_bc_r = ...
    ell = ell_ic + ell_bc_l + ell_bc_r
    ell.backward()
    opt.step()

    if i % 50 == 0:
        print(i, ell.item())

In [ ]:
plot_solution(u_hat, t0, t1)

Now for the interesting part!  We still haven't satisfied the derivatives of the PDE! How do we do this without expensive derivative approximations?  Well, we can use PyTorch's autograd to compute the derivatives for us.  For our reference, here is the formulation of the PDE again:
$$ \frac{\partial u}{\partial t} +u \frac{\partial u}{\partial x} = 0 $$

In [ ]:
x_linear = torch.linspace(-1, 1, 52)[1:-1]
t_linear = torch.linspace(t0, t1, 51)[1:]

x_pde, t_pde = torch.meshgrid(x_linear, t_linear, indexing='ij')
x_pde = x_pde.flatten()
t_pde = t_pde.flatten()
x_pde.requires_grad = True
t_pde.requires_grad = True

# what does the above do? for each time step, it creates a spatial grid we can evaluate our function at.
plt.scatter(x_pde.detach(), t_pde.detach(), s=1.0)
plt.xlabel('x')
plt.ylabel('t')

In [ ]:
u_eval = u_hat(x_pde, t_pde)

du_dx = torch.autograd.grad(u_eval, x_pde, 
                            grad_outputs=torch.ones_like(x_pde), # Shape of the output
                            create_graph=True, retain_graph=True)[0]

du_dt = torch.autograd.grad(u_eval, t_pde, 
                            grad_outputs=torch.ones_like(x_pde), # Shape of the output
                            create_graph=True, retain_graph=True)[0]

print(du_dx)
print(du_dt)

How well do we currently satisfy the PDE portion?

In [ ]:
torch.mean((du_dt + u_eval * du_dx)**2.)

Lets combine everything into one large loss function and train.

In [ ]:
# space, time for initial conditions
x_ic = torch.linspace(-1, 1, 50)
t_ic = torch.zeros_like(x)

# space, time for boundary conditions
t_bc = torch.linspace(t0, t1, 50)
x_l_bc = torch.ones_like(t_bc) * -1
x_r_bc = torch.ones_like(t_bc) * 1

# space, time for pde residual
x_linear = torch.linspace(-1, 1, 52)[1:-1]
t_linear = torch.linspace(t0, t1, 20)[1:]

x_pde, t_pde = torch.meshgrid(x_linear, t_linear, indexing='ij')
x_pde = x_pde.flatten()
t_pde = t_pde.flatten()
x_pde.requires_grad = True
t_pde.requires_grad = True

opt = torch.optim.Adam(u_hat.parameters(), lr=1e-3)

for i in range(50_000):
    # combine everything and fill out this loop!

In [ ]:
plot_solution(u_hat, t0, t1)